# Introduction to Claude Agents SDK + MCP

## 🎯 Learning Objectives

By the end of this notebook, you will:
- Understand what the Claude Agents SDK is and how it works
- Learn the difference between in-process and external MCP servers
- Create a streaming session with Claude
- Attach an MCP Filesystem server to your agent
- List available tools and invoke them programmatically
- Build a minimal "File Reader" agent

## 📚 What is the Claude Agents SDK?

The **Claude Agents SDK** is a Python library that allows you to build AI agents powered by Claude. It provides:

- 🔄 **Streaming Sessions**: Continuous conversations with Claude
- 🛠️ **Tool Integration**: Easy integration with external tools via MCP
- 🎣 **Hooks**: Intercept and control Claude's actions
- ⚡ **Performance**: In-process MCP servers with no subprocess overhead

### Key Components

1. **`query()`**: Simple one-shot streaming function
2. **`ClaudeSDKClient`**: Full-featured bidirectional client for complex workflows
3. **`ClaudeAgentOptions`**: Configuration for your agent's behavior
4. **MCP Integration**: Connect to tools, resources, and prompts

## 🔌 MCP Integration: Two Approaches

### 1. In-Process SDK Servers

Custom tools run directly in your Python application:

```python
from claude_agent_sdk import tool, create_sdk_mcp_server

@tool("greet", "Greet a user", {"name": str})
async def greet_user(args):
    return {"content": [{"type": "text", "text": f"Hello, {args['name']}!"}]}

server = create_sdk_mcp_server(name="my-tools", version="1.0.0", tools=[greet_user])
```

**Benefits**: No subprocess overhead, better performance, simpler deployment

### 2. External MCP Servers

Connect to existing MCP servers via stdin/stdout:

```python
options = ClaudeAgentOptions(
    mcp_servers={
        "filesystem": {
            "type": "stdio",
            "command": "npx",
            "args": ["@modelcontextprotocol/server-filesystem", "/path/to/dir"]
        }
    }
)
```

**Benefits**: Reuse existing MCP servers, language-agnostic tools

## 🚀 Installation

First, let's install the required packages:

In [ ]:
# Install Claude Agents SDK
!pip install -q claude-agent-sdk anthropic python-dotenv

## 🔑 Setup API Key

You'll need an Anthropic API key. Set it as an environment variable:

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Check if API key is set
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    print("⚠️  ANTHROPIC_API_KEY not found!")
    print("Set it in your .env file or:")
    print('  export ANTHROPIC_API_KEY="your-api-key"')
else:
    print("✅ API key found!")

## 📝 Example 1: Simple Query

Let's start with the simplest approach - using the `query()` function:

In [ ]:
from claude_agent_sdk import query

async def simple_query():
    """Simple one-shot query to Claude"""
    print("🤖 Asking Claude a simple question...\n")
    
    async for message in query(prompt="What is 2 + 2? Just give me the answer."):
        print(message, end="", flush=True)
    
    print("\n\n✅ Query complete!")

# Run the async function
import asyncio
await simple_query()

## 🛠️ Example 2: Creating Custom Tools (In-Process MCP)

Now let's create a custom tool using in-process MCP servers:

In [ ]:
from claude_agent_sdk import tool, create_sdk_mcp_server, ClaudeSDKClient, ClaudeAgentOptions

# Define a custom tool
@tool(
    name="get_weather",
    description="Get the current weather for a city",
    input_schema={"city": str}
)
async def get_weather(args):
    """Mock weather tool - in reality, this would call a weather API"""
    city = args.get("city", "Unknown")
    
    # Mock weather data
    weather_data = {
        "San Francisco": "Sunny, 72°F",
        "New York": "Cloudy, 68°F",
        "London": "Rainy, 61°F",
        "Tokyo": "Clear, 75°F"
    }
    
    weather = weather_data.get(city, f"Weather data not available for {city}")
    
    return {
        "content": [{
            "type": "text",
            "text": f"Current weather in {city}: {weather}"
        }]
    }

# Create MCP server with the tool
weather_server = create_sdk_mcp_server(
    name="weather-tools",
    version="1.0.0",
    tools=[get_weather]
)

print("✅ Custom weather tool created!")

### Using the Custom Tool with Claude

In [ ]:
async def use_weather_tool():
    """Use Claude with our custom weather tool"""
    
    options = ClaudeAgentOptions(
        mcp_servers={"weather": weather_server},
        allowed_tools=["mcp__weather__get_weather"],  # Note the naming convention
        max_turns=2
    )
    
    print("🤖 Claude using custom weather tool...\n")
    
    async with ClaudeSDKClient(options=options) as client:
        await client.query("What's the weather like in San Francisco and Tokyo?")
        
        async for msg in client.receive_response():
            print(msg, end="", flush=True)
    
    print("\n\n✅ Weather query complete!")

await use_weather_tool()

## 📁 Example 3: External MCP Server - Filesystem

Now let's connect to an external MCP filesystem server. First, create a sample directory with files:

In [ ]:
import os
import tempfile

# Create a temporary directory with sample files
demo_dir = tempfile.mkdtemp(prefix="claude_demo_")

# Create sample files
files = {
    "notes.txt": "This is a sample note about AI agents.\nThey are powerful tools for automation.",
    "tasks.txt": "TODO:\n- Learn Claude Agents SDK\n- Build an agent with MCP\n- Deploy to production",
    "data.txt": "Sample data: 100, 200, 300, 400, 500"
}

for filename, content in files.items():
    filepath = os.path.join(demo_dir, filename)
    with open(filepath, "w") as f:
        f.write(content)

print(f"✅ Created demo directory: {demo_dir}")
print("\nFiles created:")
for filename in files.keys():
    print(f"  - {filename}")

### Connect to Filesystem MCP Server

In [ ]:
async def use_filesystem_mcp():
    """Connect to external filesystem MCP server"""
    
    options = ClaudeAgentOptions(
        mcp_servers={
            "filesystem": {
                "type": "stdio",
                "command": "npx",
                "args": ["-y", "@modelcontextprotocol/server-filesystem", demo_dir]
            }
        },
        max_turns=3
    )
    
    print("📁 Connecting to Filesystem MCP Server...\n")
    
    async with ClaudeSDKClient(options=options) as client:
        # List available tools
        print("🔍 Listing available tools...")
        await client.query("List all available tools you have access to.")
        
        async for msg in client.receive_response():
            print(msg, end="", flush=True)
        
        print("\n\n" + "="*50 + "\n")
        
        # Read and summarize files
        print("📖 Reading and summarizing files...\n")
        await client.query(
            f"Read all .txt files in the directory and provide a brief summary of what you found."
        )
        
        async for msg in client.receive_response():
            print(msg, end="", flush=True)
    
    print("\n\n✅ Filesystem operations complete!")

await use_filesystem_mcp()

## 🎯 Example 4: Complete File Reader Agent

Let's build a complete file reader agent that:
1. Lists available files
2. Reads specific files
3. Provides summaries and insights

In [ ]:
async def file_reader_agent(directory: str, query_text: str):
    """Complete file reader agent with MCP filesystem access"""
    
    options = ClaudeAgentOptions(
        system_prompt="""You are a helpful file analysis assistant. 
        When asked to analyze files:
        1. List available files first
        2. Read relevant files
        3. Provide clear, concise summaries
        4. Highlight key information
        """,
        mcp_servers={
            "filesystem": {
                "type": "stdio",
                "command": "npx",
                "args": ["-y", "@modelcontextprotocol/server-filesystem", directory]
            }
        },
        max_turns=5  # Allow multiple tool uses
    )
    
    print(f"🤖 File Reader Agent initialized for: {directory}\n")
    print("=" * 60)
    
    async with ClaudeSDKClient(options=options) as client:
        await client.query(query_text)
        
        async for msg in client.receive_response():
            print(msg, end="", flush=True)
    
    print("\n" + "=" * 60)
    print("✅ Analysis complete!")

# Test the agent
await file_reader_agent(
    directory=demo_dir,
    query_text="Analyze all the files in this directory and tell me what tasks need to be completed."
)

## 🎣 Example 5: Using Hooks for Control

Hooks allow you to intercept and control Claude's tool usage:

In [ ]:
from claude_agent_sdk import HookMatcher

async def file_access_hook(input_data, tool_use_id, context):
    """Hook to log file access attempts"""
    tool_name = input_data.get("tool_name", "unknown")
    print(f"\n🔍 Hook triggered: {tool_name}")
    print(f"   Tool use ID: {tool_use_id}")
    
    # Allow all operations (return empty dict)
    return {}

async def agent_with_hooks():
    """Agent with hooks to monitor tool usage"""
    
    options = ClaudeAgentOptions(
        mcp_servers={
            "filesystem": {
                "type": "stdio",
                "command": "npx",
                "args": ["-y", "@modelcontextprotocol/server-filesystem", demo_dir]
            }
        },
        hooks={
            "PreToolUse": [
                HookMatcher(
                    matcher="*",  # Match all tools
                    hooks=[file_access_hook]
                )
            ]
        },
        max_turns=3
    )
    
    print("🎣 Agent with monitoring hooks enabled\n")
    
    async with ClaudeSDKClient(options=options) as client:
        await client.query("Read the notes.txt file and summarize it.")
        
        async for msg in client.receive_response():
            print(msg, end="", flush=True)
    
    print("\n\n✅ Operation complete with hooks!")

await agent_with_hooks()

## 🧹 Cleanup

In [ ]:
import shutil

# Clean up the demo directory
if os.path.exists(demo_dir):
    shutil.rmtree(demo_dir)
    print(f"✅ Cleaned up demo directory: {demo_dir}")

## 📚 Key Takeaways

1. **Simple Start**: Use `query()` for one-off tasks
2. **Custom Tools**: Create in-process MCP servers with `@tool` decorator
3. **External Tools**: Connect to existing MCP servers via stdio
4. **Streaming**: Use `ClaudeSDKClient` for bidirectional conversations
5. **Control**: Use hooks to monitor and control tool usage

## 🎯 Next Steps

- Explore combining multiple MCP servers
- Build more complex tools with error handling
- Learn about resources and prompts in MCP
- Create production-ready agents with proper configuration

## 📖 Resources

- [Claude Agents SDK Documentation](https://docs.claude.com/en/api/agent-sdk/overview)
- [MCP Specification](https://modelcontextprotocol.io/specification/)
- [GitHub Repository](https://github.com/anthropics/claude-agent-sdk-python)
- [Example Projects](https://github.com/anthropics/claude-agent-sdk-python/tree/main/examples)
